<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0728/10_1_(practice)UNET_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# implementing  custom U-net

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### standard U-Net 구현

- 주요 함수 :  
1. layers.Cropping2D(cropping=((top_crop, bottom_crop), (left_crop, right_crop)))  
2. upconv_block(inputs, filters, kernel_size=2):  
    x = layers.UpSampling2D(size=(2, 2))(inputs)  
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)


In [ ]:
img_size = (572, 572,3)
num_classes = 2

In [ ]:
def conv2d(inputs, filters):
    x = tf.keras.layers.Conv2D(filters , kernel_size=3, padding = 'valid', activation='relu')(inputs)
    return x

In [ ]:
def upconv_block(inputs, filters, kernel_size=2):
    x = layers.UpSampling2D(size=(2, 2))(inputs)
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)
    return x

In [ ]:
input = tf.keras.layers.Input(shape=[64,64,512])
x = tf.keras.layers.Cropping2D(cropping=(((, (input.shape[1]- x.shape[1])//2)), ((input.shape[1]- x.shape[1])//2, (input.shape[1]- x.shape[1])//2))(input)

print(()
# mode = tf.keras.Model(inputs = input, outputs = x
# mode.summary()

4


In [ ]:
def unet(img_size, num_classes):
    inputs = tf.keras.layers.Input(shape=img_size)
    x = conv2d(inputs, 64)
    x = conv2d(x, 64)

    add1 = x # 568,586,64

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv2d(x, 128)
    x = conv2d(x, 128)

    add2 = x # 280,280,128

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv2d(x, 256)
    x = conv2d(x, 256)

    add3 = x # 136,136,256

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv2d(x, 512)
    x = conv2d(x, 512)

    add4 = x # 64,64, 512

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv2d(x, 1024)
    x = conv2d(x, 1024)

    x = upconv_block(x, 512) # 56,56,512
    x_1 = x
    x = tf.keras.layers.CenterCrop(56,56)(add4)

    x = tf.keras.layers.Concatenate(axis=-1)([x_1, x])
    x = conv2d(x, 512)
    x = conv2d(x, 512)

    x = upconv_block(x, 256)
    x_2 = x
    x = tf.keras.layers.CenterCrop(104,104)(add3)
    x = tf.keras.layers.Concatenate(axis=-1)([x_2, x])
    x = conv2d(x, 256)
    x = conv2d(x, 256)

    x = upconv_block(x, 128)
    x_3 = x
    x = tf.keras.layers.CenterCrop(200,200)(add2)
    x = tf.keras.layers.Concatenate(axis=-1)([x_3, x])
    x = conv2d(x, 128)
    x = conv2d(x, 128)

    x = upconv_block(x, 64)
    x_4 = x
    x = tf.keras.layers.CenterCrop(392,392)(add1)
    x = tf.keras.layers.Concatenate(axis=-1)([x_4, x])
    x = conv2d(x, 64)
    x = conv2d(x, 64)
    outputs = tf.keras.layers.Conv2D(num_classes , kernel_size=1, padding = 'valid', activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [ ]:
def unet(img_size, num_classes):
    inputs = tf.keras.layers.Input(shape=img_size)
    x = conv2d(inputs, 64)
    x = conv2d(x, 64)

    add1 = x # 568,586,64

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)
    x = conv2d(x, 128)
    x = conv2d(x, 128)

    add2 = x # 280,280,128

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)
    x = conv2d(x, 256)
    x = conv2d(x, 256)

    add3 = x # 136,136,256

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)
    x = conv2d(x, 512)
    x = conv2d(x, 512)

    add4 = x # 64,64, 512

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)
    x = conv2d(x, 1024)
    x = conv2d(x, 1024)

    x = upconv_block(x, 512) # 56,56,512
    x_1 = x

    crop1 = (add4.shape[1]- x_1.shape[1])//2
    x = x = tf.keras.layers.Cropping2D(cropping=((crop1,crop1), (crop1,crop1)))(add4)
    x = tf.keras.layers.Concatenate(axis=-1)([x_1, x])
    x = conv2d(x, 512)
    x = conv2d(x, 512)

    x = upconv_block(x, 256)
    x_2 = x

    crop2 = (add3.shape[1]- x_2.shape[1])//2
    x = x = tf.keras.layers.Cropping2D(cropping=((crop2,crop2), (crop2,crop2)))(add3)
    x = tf.keras.layers.Concatenate(axis=-1)([x_2, x])
    x = conv2d(x, 256)
    x = conv2d(x, 256)

    x = upconv_block(x, 128)
    x_3 = x

    crop3 = (add2.shape[1]- x_3.shape[1])//2
    x = x = tf.keras.layers.Cropping2D(cropping=((crop3,crop3), (crop3,crop3)))(add2)
    x = tf.keras.layers.Concatenate(axis=-1)([x_3, x])
    x = conv2d(x, 128)
    x = conv2d(x, 128)

    x = upconv_block(x, 64)
    x_4 = x
    crop4 = (add1.shape[1]- x_4.shape[1])//2
    x = x = tf.keras.layers.Cropping2D(cropping=((crop4,crop4), (crop4,crop4)))(add1)
    x = tf.keras.layers.Concatenate(axis=-1)([x_4, x])
    x = conv2d(x, 64)
    x = conv2d(x, 64)
    outputs = tf.keras.layers.Conv2D(num_classes , kernel_size=1, padding = 'valid', activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [ ]:
model = unet(img_size, num_classes)
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_51 (InputLayer)          [(None, 572, 572, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_372 (Conv2D)            (None, 570, 570, 64  1792        ['input_51[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_373 (Conv2D)            (None, 568, 568, 64  36928       ['conv2d_372[0][0]']             
                                )                                                          

In [ ]:
def unet(img_size, num_classes):
    return tf.keras.Model(inputs, outputs)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 572, 572, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 570, 570, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 568, 568, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

## custom unet 구현
1. no padding -> padding
2. crop -> no crop

In [ ]:
img_size = (128, 128, 3)
num_classes = 3

In [ ]:
def conv_2d(inputs, filters):
    x = tf.keras.layers.Conv2D(filters , kernel_size=3, padding = 'same', activation='relu')(inputs)
    return x

In [ ]:
def custum_unet(img_size, num_classes):
    inputs = tf.keras.layers.Input(shape=img_size)
    x = conv_2d(inputs, 64)
    x = conv_2d(x, 64)
    add1 = x

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_2d(x, 128)
    x = conv_2d(x, 128)

    add2 = x

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_2d(x, 256)
    x = conv_2d(x, 256)

    add3 = x

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_2d(x, 512)
    x = conv_2d(x, 512)

    add4 = x # 64,64, 512

    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_2d(x, 1024)
    x = conv_2d(x, 1024)

    x = upconv_block(x, 512) # 56,56,512
    x = tf.keras.layers.Concatenate(axis=-1)([add4, x])
    x = conv_2d(x, 512)
    x = conv_2d(x, 512)

    x = upconv_block(x, 256)
    x = tf.keras.layers.Concatenate(axis=-1)([add3, x])
    x = conv_2d(x, 256)
    x = conv_2d(x, 256)

    x = upconv_block(x, 128)
    x = tf.keras.layers.Concatenate(axis=-1)([add2, x])
    x = conv_2d(x, 128)
    x = conv_2d(x, 128)

    x = upconv_block(x, 64)
    x = tf.keras.layers.Concatenate(axis=-1)([add1, x])
    x = conv_2d(x, 64)
    x = conv_2d(x, 64)
    outputs = tf.keras.layers.Conv2D(num_classes , kernel_size=1, padding = 'valid', activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [ ]:
models = custum_unet(img_size, num_classes)
models.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_253 (Conv2D)            (None, 128, 128, 64  1792        ['input_37[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_254 (Conv2D)            (None, 128, 128, 64  36928       ['conv2d_253[0][0]']             
                                )                                                          

In [ ]:
def custum_unet(img_size, num_classes):

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  36928       ['conv2d[0][0]']                 
                                )                                                             